# 15장 동적 계획법

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/python4daml/blob/main/15장.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

## 15.1 피보나치 수열 다시 살펴 보기

In [ ]:
def fib(n): 
    """n은 0보다 크거나 같은 정수라고 가정합니다.
       n의 피보나치 수열 값을 반환합니다""" 
    if n == 0 or n == 1: 
        return 1 
    else: 
        return fib(n-1) + fib(n-2) 

그림 15-2 동적 계획법을 사용한 피보나치 구현

In [ ]:
def fib_memo(n, memo = None): 
    """n은 0보다 크거나 같은 정수라고 가정하며, memo는 재귀 호출에서만 사용됩니다.
       n의 피보나치 수열 값을 반환합니다"""
    if memo == None: 
     memo = {} 
    if n == 0 or n == 1: 
        return 1 
    try: 
        return memo[n] 
    except KeyError: 
        result = fib_memo(n-1, memo) + fib_memo(n-2, memo) 
        memo[n] = result 
        return result 

def fib_tab(n): 
    """n은 0보다 크거나 같은 정수라고 가정합니다.
       n의 피보나치 수열 값을 반환합니다""" 
    tab = [1]*(n+1) #처음 두 값만 맞습니다
    for i in range(2, n + 1): 
        tab[i] = tab[i-1] + tab[i-2] 
    return tab[n] 

In [ ]:
fib_memo(120)

8670007398507948658051921

In [ ]:
fib_tab(120)

8670007398507948658051921

**손가락 운동**

In [ ]:
def make_change(coin_vals, change): 
    """coin_vals는 양수 리스트이고 coin_vals[0] = 1입니다.
       change는 양수입니다.
       동전의 합이 change가 되기 위해 필요한 최소 동전 개수를 반환합니다.
          동전은 한 번 이상 사용할 수 있습니다.
          예를 들어 make_change([1, 5, 8], 11)은 3을 반환해야 합니다."""

## 15.2 동적 계획법과 0/1 배낭 문제

그림 15-5 결정 트리를 사용해 배낭 문제 해결하기

In [ ]:
def max_val(to_consider, avail): 
    """to_consider는 물건의 리스트이고 avail은 무게라고 가정합니다.
       0/1 배낭 문제의 해에 대한 총 가치와 물건을 튜플로 반환합니다"""
    if to_consider == [] or avail == 0: 
        result = (0, ()) 
    elif to_consider[0].get_weight() > avail: 
        #오른쪽 가지만 탐색합니다
        result = max_val(to_consider[1:], avail) 
    else: 
        next_item = to_consider[0] 
        #왼쪽 가지를 탐색합니다
        with_val, with_to_take = max_val(to_consider[1:], 
                                         avail - next_item.get_weight()) 
        with_val += next_item.get_value() 
        #오른쪽 가지를 탐색합니다
        without_val, without_to_take = max_val(to_consider[1:], avail) 
        #더 나은 가지를 선택합니다
        if with_val > without_val: 
            result = (with_val, with_to_take + (next_item,)) 
        else: 
            result = (without_val, without_to_take) 
    return result

In [ ]:
class Item(object): 
    def __init__(self, n, v, w): 
        self._name = n 
        self._value = v 
        self._weight = w 
    def get_name(self): 
        return self._name 
    def get_value(self): 
        return self._value 
    def get_weight(self): 
        return self._weight 
    def __str__(self): 
        return f'<{self._name}, {self._value}, {self._weight}>' 

그림 15-6 결정 트리 기반 구현 테스트하기

In [ ]:
import random

def small_test(): 
    names = ['a', 'b', 'c', 'd'] 
    vals = [6, 7, 8, 9] 
    weights = [3, 3, 2, 5] 
    Items = [] 
    for i in range(len(vals)): 
        Items.append(Item(names[i], vals[i], weights[i])) 
    val, taken = max_val(Items, 5) 
    for item in taken: 
        print(item) 
    print('선택한 물건의 총 가치 =', val) 

def build_many_items(num_items, max_val, max_weight): 
    items = [] 
    for i in range(num_items): 
        items.append(Item(str(i), 
                          random.randint(1, max_val), 
                          random.randint(1, max_weight))) 
    return items 

def big_test(num_items, avail_weight): 
    items = build_many_items(num_items, 10, 10) 
    val, taken = max_val(items, avail_weight) 
    print('선택한 물건') 
    for item in taken: 
        print(item) 
    print('선택한 물건의 총 가치 =', val) 

In [ ]:
small_test()

<c, 8, 2>
<b, 7, 3>
선택한 물건의 총 가치 = 15


In [ ]:
big_test(10, 40)

선택한 물건
<8, 5, 1>
<6, 9, 3>
<5, 10, 3>
<4, 6, 6>
<3, 9, 7>
<2, 5, 10>
<1, 8, 1>
<0, 7, 9>
선택한 물건의 총 가치 = 59


그림 15-7 배낭 문제에 대한 동적 계획법 솔루션

In [ ]:
def fast_max_val(to_consider, avail, memo = {}): 
    """to_consider는 물건의 리스트이고 avail은 무게라고 가정합니다.
       memo는 재귀 호출에 의해 제공됩니다.
       0/1 배낭 문제의 해에 대한 총 가치와 물건을 튜플로 반환합니다""" 
    if (len(to_consider), avail) in memo: 
        result = memo[(len(to_consider), avail)] 
    elif to_consider == [] or avail == 0: 
        result = (0, ()) 
    elif to_consider[0].get_weight() > avail: 
        #오른쪽 가지만 탐색합니다
        result = fast_max_val(to_consider[1:], avail, memo) 
    else: 
        next_item = to_consider[0] 
        #왼쪽 가지를 탐색합니다
        with_val, with_to_take = \
                fast_max_val(to_consider[1:], 
                             avail - next_item.get_weight(), memo) 
        with_val += next_item.get_value() 
        #오른쪽 가지를 탐색합니다
        without_val, without_to_take = fast_max_val(to_consider[1:], 
                                                    avail, memo) 
        #더 나은 가지를 선택합니다
        if with_val > without_val: 
            result = (with_val, with_to_take + (next_item,)) 
        else: 
            result = (without_val, without_to_take) 
    memo[(len(to_consider), avail)] = result 
    return result 

In [ ]:
def big_test(num_items, avail_weight): 
    items = build_many_items(num_items, 10, 10) 
    val, taken = fast_max_val(items, avail_weight) 
    print('선택한 물건') 
    for item in taken: 
        print(item) 
    print('선택한 물건의 총 가치 =', val) 

big_test(40, 100)

선택한 물건
<39, 10, 8>
<37, 9, 5>
<36, 8, 8>
<35, 3, 1>
<33, 7, 1>
<32, 7, 4>
<31, 5, 2>
<28, 5, 1>
<27, 6, 8>
<26, 8, 1>
<25, 10, 4>
<24, 4, 3>
<23, 8, 9>
<22, 7, 1>
<21, 9, 5>
<18, 10, 3>
<14, 7, 6>
<11, 3, 1>
<9, 8, 5>
<6, 6, 1>
<4, 9, 4>
<3, 3, 1>
<2, 7, 6>
<1, 8, 6>
<0, 5, 6>
선택한 물건의 총 가치 = 172
